<h1 align="center">🧵 Quwa Apparel Sales Analysis Dashboard</h1>
<h3 align="center">Using Python & Power BI</h3>
<p align="center"><em>By Zuhayr Chowdhury</em></p>

---

This notebook presents a sales analysis of my own streetwear brand, **Quwa Apparel**, using simulated Shopify sales data. As the founder and operations lead, I wanted to dive deep into our store’s performance to uncover insights around revenue trends, product performance, and customer behavior.

The analysis is done in Python to clean, structure, and explore the data. Insights generated here will later be visualized in a Power BI dashboard to support smarter business decisions around marketing, inventory, and customer retention.

This project is a core part of my data analyst portfolio, blending real-world business ownership with technical analytics skills.


## 1. Data Loading & Preprocessing
In this section, we import the sales dataset exported from the Quwa Apparel Shopify store.
We perform essential preprocessing steps including:
- Converting date fields for time-based analysis
- Calculating total revenue and estimated profit
- Cleaning customer status fields (repeat vs. new)

These steps prepare the data for deeper analysis and visualization in later sections.


In [1]:
import pandas as pd

# Load dataset
df = pd.read_csv("quwa_sales_data.csv")

# Preview
df.head()

,Order ID,Order Date,Customer ID,Customer Name,Email,City,State,Country,Product Title,Product Category,SKU,Quantity,Unit Price,Discount Applied,Total Price,Payment Method,Is Repeat Customer
0,1000,2024-08-24,C021,Ms. Katherine Walker DDS,bradykristi@gmail.com,New Dannymouth,DC,US,Zipper Hoodie,Hoodie,QWA-H007,1,45.97,0.0,45.97,PayPal,Yes
1,1001,2024-12-02,C022,Jennifer Garcia,patricia37@yahoo.com,South John,MT,US,Slim Fit Joggers,Pants,QWA-P009,1,37.69,0.0,37.69,Credit Card,No
2,1002,2024-10-19,C004,Omar Perry,erinyoung@hotmail.com,West Stevenstad,CT,US,Oversized Street Tee,Shirt,QWA-S014,3,41.52,0.0,124.56,Shop Pay,No
3,1003,2024-05-17,C018,Madeline Welch,moorejoanna@hotmail.com,Port Kennethtown,KS,US,Classic Logo Sweater,Sweater,QWA-S004,2,61.09,0.0,122.18,Credit Card,Yes
4,1004,2024-09-06,C001,Vickie Bell,lauren73@weiss-walker.biz,South Kathleen,NC,US,Oversized Hoodie,Hoodie,QWA-H001,1,36.21,0.0,36.21,Shop Pay,No


In [2]:
# Convert 'Order Date' to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])

# Create 'Order Month' for grouping
df['Order Month'] = df['Order Date'].dt.to_period('M')

# Make sure all price columns are numeric
df['Total Price'] = pd.to_numeric(df['Total Price'], errors='coerce')
df['Unit Price'] = pd.to_numeric(df['Unit Price'], errors='coerce')
df['Discount Applied'] = pd.to_numeric(df['Discount Applied'], errors='coerce')


In [3]:
# Revenue (already total price, but we double-check)
df['Calculated Revenue'] = (df['Unit Price'] * df['Quantity']) - df['Discount Applied']

# Profit Margin estimate (assume cost = 60% of price)
df['Estimated Profit'] = df['Calculated Revenue'] * 0.4

# Flag repeat customers
df['Is Repeat Customer'] = df['Is Repeat Customer'].str.strip().str.lower()


## 2. Sales Performance & Customer Insights
This section focuses on analyzing key business metrics for Quwa Apparel, based on the cleaned Shopify sales data.

We explore:
- **Monthly Revenue Trends**: Identifying how sales have varied over time, including seasonal spikes from sales events like Eid and Black Friday.
- **Product Performance**: Ranking product categories and individual items based on revenue and quantity sold to uncover top-performing inventory.
- **Customer Segmentation**: Breaking down total revenue from new vs. repeat customers to assess retention and customer loyalty.

These insights help inform inventory decisions, marketing strategy, and potential growth opportunities for the brand.


In [8]:
# 1. Monthly Revenue
monthly_revenue = df.groupby('Order Month')['Calculated Revenue'].sum().reset_index()
print("📅 Monthly Revenue")
print(monthly_revenue)

# 2. Revenue by Category
category_revenue = df.groupby('Product Category')['Calculated Revenue'].sum().sort_values(ascending=False)
print("\n📦 Revenue by Category")
print(category_revenue)

# 3. Top 10 Products
top_products = df.groupby('Product Title')['Quantity'].sum().sort_values(ascending=False).head(10)
print("\n🔥 Top 10 Selling Products (by Quantity)")
print(top_products)

# 4. New vs Repeat Customers
repeat_revenue = df[df['Is Repeat Customer'] == 'yes']['Calculated Revenue'].sum()
new_revenue = df[df['Is Repeat Customer'] == 'no']['Calculated Revenue'].sum()
print("\n👥 Customer Revenue Breakdown")
print(f"Repeat Customers: ${repeat_revenue:.2f}")
print(f"New Customers: ${new_revenue:.2f}")

📅 Monthly Revenue
   Order Month  Calculated Revenue
0      2024-01             1228.10
1      2024-02              526.59
2      2024-03             1017.71
3      2024-04              640.98
4      2024-05              560.11
5      2024-06             1076.75
6      2024-07              352.84
7      2024-08             1104.98
8      2024-09              392.17
9      2024-10              124.56
10     2024-11             1957.27
11     2024-12             1353.11

📦 Revenue by Category
Product Category
Hoodie     3881.67
Shirt      2613.90
Pants      1942.18
Sweater    1897.42
Name: Calculated Revenue, dtype: float64

🔥 Top 10 Selling Products (by Quantity)
Product Title
Oversized Hoodie          36
Oversized Street Tee      29
Minimalist Sweater        25
Zipper Hoodie             23
Classic Logo Hoodie       20
Slim Fit Joggers          14
Relaxed Cargo Pants       14
Streetwear Track Pants    13
Essential Tee             12
Minimal Logo Tee          12
Name: Quantity, dtype: in